In [221]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.signal import convolve2d
from scipy.optimize import curve_fit
from scipy.optimize import brute
# import cv2
import torch
import json
import csv
from functions.all_knots_functions import *
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from sklearn.model_selection import train_test_split
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import collections
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [222]:
knot = 'standard_16'
filename = f'..\data\data_{knot}.csv'
with open(filename, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        # Deserialize the JSON string back to a list
        data_list = json.loads(row[0])
        # Convert the list back to a NumPy array if needed
        data_array = np.array(data_list)
        print(data_array.shape)

(108, 3)
(159, 3)


In [223]:
Nx, Ny, Nz = data_array[1]
number = data_array[0][0]
points_list = data_array[2:]

# Initialize a 3D array with zeros
dots_3d = np.zeros((Nx, Ny, Nz), dtype=int)

# Set the specified coordinates to 1
for x, y, z in points_list:
    dots_3d[x, y, z] = 1

In [224]:
knot_types = {
	'standard_14': 0,  # 1
	'standard_16': 1,  # 2
	'standard_18': 2,  # 3
	'30both': 3,  # 4
	'30oneZ': 4,  # 5
	'optimized': 5,  # 6
	'pm_03_z': 6,  # 7
	'4foil': 7,  # 8
	'6foil': 8,  # 9
	'stand4foil': 9,  # 10
	'30oneX': 10,  # 11

}
knots = [
	'standard_14', 'standard_16', 'standard_18', '30both', '30oneZ',
	'optimized', 'pm_03_z', '4foil', '6foil', 'stand4foil',
	'30oneX'
]
num_classes = len(knots)
X_list = []
Y_list = []
for knot in knots:
    filename = f'..\data\data_{knot}.csv'
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            # Deserialize the JSON string back to a list
            data_list = json.loads(row[0])
            # Convert the list back to a NumPy array if needed
            data_array = np.array(data_list)
            # Initialize a 3D array with zeros
            # dots_3d = np.zeros((Nx, Ny, Nz), dtype=int)
            # # Set the specified coordinates to 1
            # for x, y, z in points_list:
            #     dots_3d[x, y, z] = 1
            # X_list.append(dots_3d)
            X_list.append(data_array)
            Y_list.append(knot_types[knot])


In [225]:
print(X_list)

[array([[ 0,  0,  0],
       [64, 64, 64],
       [16, 31, 32],
       [16, 31, 33],
       [16, 32, 31],
       [16, 33, 29],
       [16, 33, 30],
       [16, 33, 28],
       [16, 34, 26],
       [16, 34, 27],
       [16, 35, 25],
       [16, 36, 24],
       [16, 36, 23],
       [17, 30, 35],
       [17, 30, 36],
       [17, 31, 34],
       [17, 36, 22],
       [17, 37, 21],
       [18, 29, 37],
       [18, 38, 20],
       [18, 38, 19],
       [18, 28, 38],
       [19, 28, 39],
       [19, 39, 18],
       [21, 26, 40],
       [21, 40, 17],
       [22, 40, 16],
       [23, 41, 15],
       [25, 41, 14],
       [26, 25, 40],
       [28, 26, 39],
       [28, 41, 14],
       [30, 27, 38],
       [30, 41, 15],
       [31, 28, 37],
       [31, 30, 26],
       [31, 30, 27],
       [31, 30, 28],
       [31, 31, 29],
       [31, 40, 16],
       [32, 28, 23],
       [32, 28, 24],
       [32, 28, 36],
       [32, 29, 25],
       [32, 31, 30],
       [32, 31, 31],
       [32, 31, 32],
       [32, 

In [226]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, X_list, y_list, desired_res):

        # self.dataset = dataset
        self.X = X_list
        self.y = y_list
        self.desired_res = desired_res

    def __len__(self):

        return len(self.X)

    def __getitem__(self, idx):
        """
        Retrieves the sample and its label at the given index.

        Args:
            idx: Index of the sample to retrieve.

        Returns:
            A tuple (sample, label).
        """
        Nx, Ny, Nz = self.X[idx][1]
        # number = data_array[0][0]
        points_list = self.X[idx][2:]
        # print(points_list)
        dots_3d = np.zeros(self.desired_res, dtype=int)
        if self.desired_res != (Nx, Ny, Nz):
            scale_x = self.desired_res[0] / Nx
            scale_y = self.desired_res[1] / Ny
            scale_z = self.desired_res[2] / Nz
            points_list = np.rint(points_list * np.array([scale_x, scale_y, scale_z])).astype(int)
        # Initialize a 3D array with zeros
        # # Set the specified coordinates to 1
        for x, y, z in points_list:
            try: dots_3d[x, y, z] = 1
            except IndexError: continue

        # X_list.append(dots_3d)
        data = torch.tensor(dots_3d).reshape(1, *self.desired_res).float()
        label = torch.tensor(self.y[idx])
        # Here, you can add custom processing to your data
        # For example, adding a custom transformation, processing data, etc.

        return data, label

    def getitem_slice(self, idx):
        """
        Retrieves the sample and its label at the given index.

        Args:
            idx: Index of the sample to retrieve.

        Returns:
            A tuple (sample, label).
        """
        Nx_ar, Ny_ar, Nz_ar = np.array([self.X[i][1] for i in range(idx.start, idx.stop)]).T
        # number = data_array[0][0]
        points_list = [self.X[i][2:] for i in range(idx.start, idx.stop)]
        # print(points_list)
        dots_3d = np.zeros(((idx.stop-idx.start), *self.desired_res), dtype=int)
        for i in range(len(Nx_ar)):
            if self.desired_res != (Nx_ar[i], Ny_ar[i], Nz_ar[i]):
                scale_x = self.desired_res[0] / Nx_ar[i]
                scale_y = self.desired_res[1] / Ny_ar[i]
                scale_z = self.desired_res[2] / Nz_ar[i]
                points_list[i] = np.rint(points_list[i] * np.array([scale_x, scale_y, scale_z])).astype(int)
            # Initialize a 3D array with zeros
            # # Set the specified coordinates to 1
            for x, y, z in points_list[i]:
                try: dots_3d[i, x, y, z] = 1
                except IndexError: continue

        # X_list.append(dots_3d)
        data = torch.tensor(dots_3d).reshape(-1, 1, *self.desired_res).float()
        label = torch.tensor(self.y[idx]).float()
        # Here, you can add custom processing to your data
        # For example, adding a custom transformation, processing data, etc.

        return data, label


In [227]:
desired_res = (32, 32, 32)
XY_dataset = CustomDataset(X_list, Y_list, desired_res=desired_res)
XY_dataset[1][0].shape

torch.Size([1, 32, 32, 32])

In [228]:
# Define the proportions
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2
total_length = len(XY_dataset)
train_length = int(total_length * train_ratio)
val_length = int(total_length * val_ratio)
test_length = total_length - train_length - val_length

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(XY_dataset, [train_length, val_length, test_length])

print(f"Training Set: {len(train_dataset)} samples")
print(f"Validation Set: {len(val_dataset)} samples")
print(f"Test Set: {len(test_dataset)} samples")

Training Set: 13 samples
Validation Set: 4 samples
Test Set: 5 samples


In [229]:
# X_np = np.array(X_list)
# y_np = np.array(Y_list)
# # X_torch = torch.tensor(X_np).reshape(-1,1,Nx, Ny, Nz).float()
# X_torch = torch.tensor(X_np).float()
# y_torch_list = torch.tensor(y_np)
# y_torch = F.one_hot(y_torch_list.long(), num_classes=num_classes).float()
# print(X_torch.shape, y_torch.shape)

In [230]:
# dots_bound = [
#     [0, 0, 0],
#     [Nx, Ny, Nz],
# ]
# # pl.plotDots(dots_init, dots_init, color='black', show=True, size=10)
# pl.plotDots(points_list, dots_bound, color='black', show=False, size=10)

In [231]:
def conv_stage(layer_configs):
    layers = []
    for config in layer_configs:
        in_channels, out_channels, kernel_size, stride, padding = config
        layers.append(nn.Conv3d(in_channels, out_channels, kernel_size, stride, padding))
        layers.append(nn.BatchNorm3d(out_channels))
        layers.append(nn.ReLU(inplace=True))

    return nn.Sequential(*layers)

def create_pooling_layer(config):
    if config is None:
        return None
    kernel_size, stride, padding = config
    return nn.MaxPool3d(kernel_size=kernel_size, stride=stride, padding=padding)

class Classifier3D(nn.Module):
    def __init__(self, stages, pooling_configs, num_classes=11, desired=desired_res):
        super(Classifier3D, self).__init__()

        self.features = nn.Sequential()
        self.desired = desired_res

        for i, stage in enumerate(stages):
            # Add the convolutional stage
            self.features.add_module(f"stage_{i}", conv_stage(stage))

            # Add a custom MaxPooling layer after each stage based on the pooling configuration
            if i < len(pooling_configs):
                pool_layer = create_pooling_layer(pooling_configs[i])
                if pool_layer:
                    self.features.add_module(f"pool_{i}", pool_layer)


        # Calculate the size of the flattened features after the conv layers
        self._to_linear = None
        self._get_conv_output((1, *self.desired))

        # Fully connected layers
        self.fc1 = nn.Linear(self._to_linear, 256)
        # self.fc2 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(256, num_classes)

    def _get_conv_output(self, shape):
        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))
        output_feat = self.features(input)
        self._to_linear = int(np.prod(output_feat.size()[1:]))

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1) # Flatten the output
        x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        x = self.fc2(x)
        x = nn.Softmax(1)(x)
        return x


In [232]:
# in_channels, out_channels, kernel_size, stride, padding
stages = [
    [(1, 32, 3, 1, 1), (32, 32, 3, 1, 1)],
    # [(32, 64, 3, 1, 1), (64, 64, 3, 1, 1)],
    [(32, 64, 3, 1, 1)]
]

# Define pooling configurations: (kernel_size, stride, padding)
# Set to 'None' for stages where no pooling is desired
# kernel_size, stride, padding
pooling_configs = [
    (4, 4, 0),  # Pooling after the first stage
    # (3, 2, 1),  # Pooling after the second stage
    None        # No pooling after the third stage
]

model = Classifier3D(stages, pooling_configs, num_classes=11).to(device)
model.initialize_weights()

In [ ]:
# print(model._to_linear, 512 * 16 * 16)
dots_3d_toch_batch = XY_dataset.getitem_slice(slice(1, 3, None))[0].to(device)
print(dots_3d_toch_batch.shape)
model(dots_3d_toch_batch)

In [ ]:
def loop_train(model, train_loader, criterion, optimizer):
    model.train()  # Set the model to training mode
    total_loss = 0
    for i, (inputs, targets) in enumerate(train_loader, 1):  # Start enumeration from 1
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()  # Clear the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights
        total_loss += loss.item()  # Accumulate the loss

    return total_loss / len(train_loader)  # Return the average loss


def loop_test(model, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    with torch.no_grad():  # No need to track the gradients
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, targets)  # Compute the loss
            total_loss += loss.item()  # Accumulate the loss
    return total_loss / len(test_loader)  # Return the average loss


def plot_losses(train_losses, test_losses, decimals=3):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses,
             label=f'Train Loss {train_losses[-1]: .{decimals}f} (min: {min(train_losses): .{decimals}f})')
    plt.plot(test_losses, label=f'Test Loss {test_losses[-1]: .{decimals}f} (min: {min(test_losses): .{decimals}f})')
    plt.title('Training and Testing Losses Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


In [234]:
hyperparams = {
    'learning_rate': 1e-4,  # Control dropout rate
    'patience': 15,  # Number of epochs between learning rate decay
    'factor': 0.2,  # Multiplicative factor of learning rate decay
    'batch_size': 3
}
# criterion = nn.MSELoss().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams['learning_rate'])
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=hyperparams['factor'], patience=hyperparams['patience'],
                              verbose=True)

num_epochs = 20
print_every = 1

train_losses = []
val_losses = []
train_loader = DataLoader(train_dataset, batch_size=hyperparams['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=hyperparams['batch_size'])
test_loader = DataLoader(test_dataset, batch_size=hyperparams['batch_size'])
for epoch in trange(num_epochs, desc="Progress"):
    train_losses.append(loop_train(model, train_loader, criterion, optimizer))
    val_losses.append(loop_test(model, val_loader, criterion))

    scheduler.step(val_losses[-1])

    if (epoch + 1) % print_every == 0:
        print(f'Epoch {epoch}: Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}')

    # # Save model and losses every 50 epochs
    # if (epoch + 1) % 100 == 0:
    #     # Save the model state
    #     print(f'model_epoch_{epoch + 1}.pth was saved')
    #     name = (
    #         f'batch={params["batch_size"]}_lr={hyperparams["learning_rate"]}_drop={hyperparams["dropout_rate"]}'
    #         f'_{name_extra}_'
    #     )
    #     torch.save(model.state_dict(), f'model_epoch_{epoch + 1}_{name}.pth')
    #     # Save losses
    #     with open(f'losses_epoch_{epoch + 1}_{name}.txt', 'w') as f:
    #         f.write(f'Train Losses: {train_losses}\n')
    #         f.write(f'Validation Losses: {val_losses}\n')


Progress:   5%|▌         | 1/20 [01:08<21:37, 68.29s/it]

Epoch 0: Train Loss: 2.3953, Val Loss: 2.4044


Progress:  10%|█         | 2/20 [01:58<17:13, 57.44s/it]

Epoch 1: Train Loss: 2.2391, Val Loss: 2.4159


Progress:  10%|█         | 2/20 [03:59<35:57, 119.85s/it]


KeyboardInterrupt: 

In [ ]:
predictions = model(X_torch.to(device))
predictions

In [ ]:
print()
_, predicted_labels = torch.max(predictions, 1)
_, true_class_labels = torch.max(y_torch.to(device), 1)
# print(predicted_labels)
# print(y_torch)
correct_predictions = torch.sum(predicted_labels == true_class_labels).item()

print(f"Number of correct predictions: {correct_predictions}")